## Core SQL Expression Language
- The SQL Expression system builds upon Table Metadata in order to compose SQL statements in Python.

- We build Python objects that represent individual SQL strings (statements) we'd send to the database.

- These objects are composed of other objects that each represent some unit of SQL, like a comparison, a SELECT statement, a conjuction such as AND or OR. 

- We work with these objects in Python, which are then converted to strings when we "execute" them (as well as if we print them)

- SQL expressions in bot Core and ORM variants rely heavily on the "method chaining" programming patter. 

In [3]:
from sqlalchemy import MetaData, Table, Column, String, Integer
from sqlalchemy import create_engine

In [4]:
metadata = MetaData()
user_table = Table(
    "user_account",
    metadata,
    Column("id", Integer, primary_key=True),
    Column("username", String(50)),
    Column("fullname", String(50)),
)

engine = create_engine("sqlite://")
with engine.begin() as conn:
    metadata.create_all(conn)

In [5]:
# as we saw earlier, Table has a collection of Column objects,
# which we can access via table.c.<columnname>

user_table.c.username

Column('username', String(length=50), table=<user_account>)

In [6]:
# Column is part of a class known as "ColumnElement",
# which exhibit custom Python expression behavior.

user_table.c.username == "spongebob"

In [7]:
# They are objects that can be **compiled** into a SQL string.   This
# process occurs when they are part of a statement to be executed.  It
# also can be viewed for debugging purposes by calling str() on the object.
str(user_table.c.username == "spongebob")

'user_account.username = :username_1'

In [9]:
# For more fine-grained inspection of the compilation process, the .compile()
# method provides the compiled form of the statement.   This includes the
# string statement itself, as well as the parameter values, which will
# have been taken from the literal values present in the statement.
compiled = (user_table.c.username == "spongebob").compile()
compiled.string, compiled.params

('user_account.username = :username_1', {'username_1': 'spongebob'})

In [10]:
### title:: Building bigger SQL constructs.
# ColumnElements are the basic building block of SQL statement objects.
# To compose more complex criteria, and_() and or_() for example provide the
# basic conjunctions of AND and OR.

from sqlalchemy import and_, or_

print(
    or_(
        user_table.c.username == "spongebob",
        user_table.c.username == "patrick"
    )
)

user_account.username = :username_1 OR user_account.username = :username_2


In [11]:
print(
    and_(
        user_table.c.fullname == "spongebob squarepants",
        or_(
            user_table.c.username == "spongebob",
            user_table.c.username == "patrick",
        )
    )
)

user_account.fullname = :fullname_1 AND (user_account.username = :username_1 OR user_account.username = :username_2)


In [12]:
### title:: More Operators

# comparison operators =, !=, >, <, >=, =<, between()

print(and_(
    user_table.c.id >= 5,
    user_table.c.fullname.between('m', 'z'),
    user_table.c.fullname != 'plankton'
))

user_account.id >= :id_1 AND user_account.fullname BETWEEN :fullname_1 AND :fullname_2 AND user_account.fullname != :fullname_3


In [13]:
# Compare to None produces IS NULL / IS NOT NULL

print(and_(
    user_table.c.username != None,
    user_table.c.fullname == None
))

user_account.username IS NOT NULL AND user_account.fullname IS NULL


In [14]:
# Operators may also be type sensitive.
# "+" with numbers means "addition"....

print(user_table.c.id + 5)

user_account.id + :id_1


In [15]:
# ...with strings it means "string concatenation"

print(user_table.c.fullname + " Jr.")

user_account.fullname || :fullname_1


In [16]:
# the IN operator generates a special placeholder that will be filled
# in when the statement is executed
criteria = user_table.c.username.in_(["sandy", "squidward", "spongebob"])
print(criteria)

user_account.username IN ([POSTCOMPILE_username_1])


In [17]:
# When it is executed, bound parameters are generated as seen here
print(criteria.compile(compile_kwargs={'render_postcompile': True}))

user_account.username IN (:username_1_1, :username_1_2, :username_1_3)


In [18]:
# when given an empty collection, the placeholder generates a SQL
# subquery that represents an "empty set"

criteria = user_table.c.username.in_([])
print(criteria.compile(compile_kwargs={'render_postcompile': True}))

user_account.username IN (NULL) AND (1 != 1)


In [19]:
### title:: Working with INSERT and SELECT Statements
# we can insert data using the insert() construct

insert_stmt = user_table.insert().values(
    username="spongebob", fullname="Spongebob Squarepants"
)

with engine.begin() as connection:
    connection.execute(insert_stmt)

In [20]:
# The insert() statement, when not given values(), will generate the VALUES
# clause based on the list of parameters that are passed to execute().

with engine.begin() as connection:
    connection.execute(
        user_table.insert(), {"username": "sandy", "fullname": "Sandy Cheeks"}
    )

    # this format also accepts an "executemany" style that the DBAPI can optimize
    connection.execute(
        user_table.insert(),
        [
            {"username": "patrick", "fullname": "Patrick Star"},
            {"username": "squidward", "fullname": "Squidward Tentacles"},
        ],
    )

In [21]:
# select() is used to produce any SELECT statement.

from sqlalchemy import select

with engine.connect() as connection:
    select_stmt = (
        select(user_table.c.username, user_table.c.fullname).
        where(user_table.c.username == "spongebob")
    )
    result = connection.execute(select_stmt)
    for row in result:
        print(row)

('spongebob', 'Spongebob Squarepants')


In [22]:
# select all columns from a table

with engine.connect() as connection:
    select_stmt = select(user_table)
    connection.execute(select_stmt).all()

In [23]:
# specify WHERE and ORDER BY

with engine.connect() as connection:
    select_stmt = select(user_table).where(
        or_(
            user_table.c.username == "spongebob",
            user_table.c.username == "sandy",
        )
    ).order_by(user_table.c.username)
    connection.execute(select_stmt).all()

In [24]:
# specify multiple WHERE, will be joined by AND

with engine.connect() as connection:
    select_stmt = (
        select(user_table)
        .where(user_table.c.username == "spongebob")
        .where(user_table.c.fullname == "Spongebob Squarepants")
        .order_by(user_table.c.username)
    )
    connection.execute(select_stmt).all()

In [26]:
with engine.connect() as connection:
    result = connection.execute(
        select(user_table.c.fullname).where(user_table.c.username == 'spongebob')
    )
    print(result.one()[0])
    

Spongebob Squarepants


In [27]:
# if there are no rows, or many rows, it raises an error.

with engine.connect() as connection:
    result = connection.execute(
    select(user_table.c.fullname).order_by(user_table.c.username)
    )
    print(result.one()[0])

MultipleResultsFound: Multiple rows were found when exactly one was required

In [28]:
with engine.connect() as connection:
    result = connection.execute(
    select(user_table).where(user_table.c.username == 'nonexistent')
    )
    print(result.one_or_none())

None


In [29]:
# result objects now support slicing at the result level.   We can SELECT
# some rows, and change the ordering and/or presence of columns after the
# fact using the .columns() method:
with engine.connect() as connection:
    result = connection.execute(
        select(user_table).order_by(user_table.c.username)
    )
    for fullname, username in result.columns("fullname", "username"):
        print(f"{fullname} {username}")

Patrick Star patrick
Sandy Cheeks sandy
Spongebob Squarepants spongebob
Squidward Tentacles squidward


In [30]:
# a single column from the results can be delivered without using
# rows by applying the .scalars() modifier.   This accepts an optional
# column name, or otherwise assumes the first column:

with engine.connect() as connection:
    result = connection.execute(
        select(user_table).order_by(user_table.c.username)
    )
    for fullname in result.scalars("fullname"):
        print(fullname)

Patrick Star
Sandy Cheeks
Spongebob Squarepants
Squidward Tentacles


In [31]:
### title:: Working with UPDATE and DELETE Statements
# The update() construct is very similar to insert().  We can specify
# values(), which here refers to the SET clause of the UPDATE statement

with engine.begin() as connection:
    update_stmt = (
        user_table.update()
        .values(fullname="Patrick Star")
        .where(user_table.c.username == "patrick")
    )

    result = connection.execute(update_stmt)

In [32]:
# Like INSERT, it can also generate the SET clause based on the given
# parameters

with engine.begin() as connection:
    update_stmt = (
        user_table.update()
        .where(user_table.c.username == "patrick")
    )

    result = connection.execute(update_stmt, {"fullname": "Patrick Star"})

In [33]:
# the update().values() method has an extra capability in that it can
# accommodate arbitrary SQL expressions as well

with engine.begin() as connection:
    update_stmt = (
        user_table.update().
        values(
            fullname=user_table.c.username + " " + user_table.c.fullname
        )
    )

    result = connection.execute(update_stmt)

In [34]:
# and this is a DELETE

with engine.begin() as connection:
    delete_stmt = user_table.delete().where(user_table.c.username == "patrick")

    result = connection.execute(delete_stmt)